Autor: Natalia Betancur Herrera

Fecha: 6/10/2025

# Laboratorio Análisis de las 5V del Big Data con PySpark en Google Colab


##Introducción

En este laboratorio exploraremos cómo Apache Spark, a través de su interfaz PySpark, nos permite analizar grandes volúmenes de datos de manera distribuida, eficiente y escalable. Trabajaremos dentro de Google Colab, un entorno gratuito y basado en la nube que facilita la ejecución de código en Python sin necesidad de instalaciones locales, aprovechando los recursos computacionales de Google.

El objetivo principal de este laboratorio es comprender y aplicar los conceptos de las 5V del Big Data (Volumen, Velocidad, Variedad, Veracidad y Valor) sobre un conjunto de datos real. Cada V representa una característica esencial que define los retos y oportunidades del procesamiento de grandes volúmenes de información:

Durante el desarrollo del laboratorio, aprenderás a:

* Configurar un entorno de PySpark en Google Colab.

* Cargar y explorar un dataset.

* Aplicar funciones distribuidas de transformación, limpieza y análisis.

* Evaluar las 5V del Big Data utilizando código y métricas interpretativas.

Al finalizar, habrás adquirido una comprensión práctica de cómo PySpark transforma el análisis de datos a gran escala, permitiéndote abordar problemas complejos en educación, salud, industria o sostenibilidad con herramientas de procesamiento masivo.



### 1. Instalación y configuración del entorno

In [1]:
!pip install pyspark --quiet

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd

In [3]:
# Crear la sesión de Spark
spark = SparkSession.builder \
    .appName("Analisis_5V_BigData") \
    .getOrCreate()

spark

### 2. Cargar el dataset

In [5]:
# Clonar el repositorio
!git clone https://github.com/NataliaBetancurH/pyspark-tutorial.git

fatal: destination path 'pyspark-tutorial' already exists and is not an empty directory.


In [13]:
# Cargar los datos
df = spark.read.csv("/content/pyspark-tutorial/dataset_educativo.csv", header=True, inferSchema=True)

In [16]:
# Verificar tipo y estructura
print(" Entorno PySpark listo")
print(f"Tipo de objeto df: {type(df)}")
df.show(5)

 Entorno PySpark listo
Tipo de objeto df: <class 'pyspark.sql.dataframe.DataFrame'>
+---+---------------+---------+------------+
| ID|   Departamento|   Genero|       Grado|
+---+---------------+---------+------------+
|  1|      Santander|Masculino|    Primaria|
|  2|      Santander|        F|  Secundaria|
|  3|Valle del Cauca| Femenino|Bachillerato|
|  3|Valle del Cauca| Femenino|Bachillerato|
|  3|Valle del Cauca| Femenino|Bachillerato|
+---+---------------+---------+------------+
only showing top 5 rows



### Analizar las 5 V del Big Data

Volumen

El volumen hace referencia a la cantidad de datos que una organización debe manejar, almacenar o analizar. A medida que el volumen crece, los métodos tradicionales (como Excel) se vuelven insuficientes, y herramientas distribuidas como PySpark resultan más eficientes.

Ejemplo de análisis:
Ver cuántos registros (filas) y cuántas variables (columnas) tiene el dataset.

In [17]:
# Contar filas y columnas
num_filas = df.count()
num_columnas = len(df.columns)

print("V de Volumen")
print(f"El dataset contiene {int(num_filas):,} registros y {num_columnas} columnas.\n")


V de Volumen
El dataset contiene 102 registros y 4 columnas.



In [18]:
# Ver estructura y tipos de datos
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Grado: string (nullable = true)



In [19]:
# Contar valores nulos (volumen de datos)
nulos = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
print("\n Conteo de valores nulos:")
nulos.show()


 Conteo de valores nulos:
+---+------------+------+-----+
| ID|Departamento|Genero|Grado|
+---+------------+------+-----+
|  0|           0|     2|    1|
+---+------------+------+-----+



In [20]:
# Mostrar ejemplo de datos
df.show(5)

+---+---------------+---------+------------+
| ID|   Departamento|   Genero|       Grado|
+---+---------------+---------+------------+
|  1|      Santander|Masculino|    Primaria|
|  2|      Santander|        F|  Secundaria|
|  3|Valle del Cauca| Femenino|Bachillerato|
|  3|Valle del Cauca| Femenino|Bachillerato|
|  3|Valle del Cauca| Femenino|Bachillerato|
+---+---------------+---------+------------+
only showing top 5 rows



Interpretación

* Mide cuánto se puede procesar con Spark.
* Los valores nulos indican volumen no aprovechable.
* PySpark puede procesar millones de filas distribuidas en memoria y disco sin problemas de rendimiento.

Velocidad

La velocidad mide la rapidez con la que los datos se generan, procesan o actualizan. En Big Data, la información llega en tiempo real o en grandes lotes continuos (por ejemplo, sensores IoT, transacciones financieras o logs web).

Ejemplo de análisis:
Medir el tiempo que tarda PySpark en ejecutar una operación, como calcular un promedio.

In [26]:
import time

print(" V de Velocidad")

# Medir tiempo de carga y procesamiento
inicio_conteo = time.time()
total_registros = df.count()
fin_conteo = time.time()

print(f" Tiempo de conteo de registros: {fin_conteo - inicio_conteo:.4f} segundos")
print(f" Total de registros procesados: {total_registros}\n")


 V de Velocidad
 Tiempo de conteo de registros: 0.4062 segundos
 Total de registros procesados: 102



In [27]:
# Medir tiempo de una agregación simple: contar personas por Departamento
inicio_agg = time.time()
df.groupBy("Departamento").agg(F.count("ID").alias("Total_por_Departamento")).show(5)
fin_agg = time.time()

print(f" Tiempo de agregación por Departamento: {fin_agg - inicio_agg:.4f} segundos\n")

+---------------+----------------------+
|   Departamento|Total_por_Departamento|
+---------------+----------------------+
|   Cundinamarca|                    11|
|      Antioquia|                    13|
|      Santander|                    20|
|      Atlántico|                     8|
|Valle del Cauca|                    18|
+---------------+----------------------+
only showing top 5 rows

 Tiempo de agregación por Departamento: 0.7417 segundos



In [36]:
def eliminar_duplicados_por_id(dataframe):
    """
    Elimina filas duplicadas basadas en la columna 'ID'.
    Conserva solo el primer registro de cada ID.
    """
    df_sin_duplicados = dataframe.dropDuplicates(["ID"])
    return df_sin_duplicados

# Aplicamos la función
df_limpio = eliminar_duplicados_por_id(df)
print(" Duplicados eliminados por ID.")

 Duplicados eliminados por ID.


In [40]:
df.groupBy("Genero").count().show()

+---------+-----+
|   Genero|count|
+---------+-----+
|       Fe|   16|
|        F|   19|
|     NULL|    2|
|        M|   18|
| Femenino|   24|
|Masculino|   23|
+---------+-----+



In [47]:
def estandarizar_genero(texto):
    """
    Corrige las variantes comunes del género:
    F, FEM, FEMENINA → FEMENINO
    M, MASC, MASCULINA → MASCULINO
    """
    if texto is None:
        return None

    texto = texto.upper().strip()
    if texto in ["F", "FE", "FEM", "FEMENINA", "FEMENINO"]:
        return "FEMENINO"
    elif texto in ["M", "MA", "MASC", "MASCULINA", "MASCULINO"]:
        return "MASCULINO"
    else:
        return texto

# Registrar UDF
udf_estandarizar_genero = F.udf(estandarizar_genero, StringType())

# Aplicar a la columna de género
df_limpio = df_limpio.withColumn("Genero", udf_estandarizar_genero(F.col("Genero")))


In [48]:
df_limpio.groupBy("Genero").count().show()

+---------+-----+
|   Genero|count|
+---------+-----+
| FEMENINO|   57|
|     NULL|    2|
|MASCULINO|   41|
+---------+-----+



In [49]:
df_sin_nulls = df_limpio.na.drop()

In [50]:
df_sin_nulls.groupBy("Genero").count().show()

+---------+-----+
|   Genero|count|
+---------+-----+
| FEMENINO|   56|
|MASCULINO|   41|
+---------+-----+



Interpretación

* La velocidad se mide por el tiempo que tarda Spark en ejecutar tareas comunes: conteo, agregación, filtrado, etc.
* Incluso con datasets grandes, PySpark mantiene tiempos bajos gracias al procesamiento en memoria y en paralelo.
* Si repites el código varias veces, notarás que Spark cachea resultados, reduciendo el tiempo en ejecuciones posteriores.

Variedad

La variedad representa los diferentes tipos y formatos de datos que se manejan:
estructurados (tablas), semiestructurados (JSON, XML) y no estructurados (texto, imágenes, audio, video). En PySpark, la variedad se aborda con diferentes APIs (Spark SQL, Spark Streaming, MLlib, etc.).

Ejemplo de análisis:
Identificar los tipos de datos en el dataset (numéricos, texto, categóricos etc).

In [29]:
print(" V de Variedad")

# Ver tipos de datos
df.printSchema()

 V de Variedad
root
 |-- ID: integer (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Grado: string (nullable = true)



In [30]:
# Mostrar columnas categóricas y numéricas
categoricas = [c for (c, t) in df.dtypes if t == "string"]
numericas = [c for (c, t) in df.dtypes if t in ["int", "double", "float"]]

print(f"Columnas categóricas: {categoricas}")
print(f"Columnas numéricas: {numericas}\n")

Columnas categóricas: ['Departamento', 'Genero', 'Grado']
Columnas numéricas: ['ID']



In [33]:
# Analizar distribución de una variable categórica
df.groupBy("GENERO").count().show()


+---------+-----+
|   GENERO|count|
+---------+-----+
|       Fe|   16|
|        F|   19|
|     NULL|    2|
|        M|   18|
| Femenino|   24|
|Masculino|   23|
+---------+-----+



Interpretación

* Spark maneja datos heterogéneos fácilmente dentro del mismo DataFrame.
* Permite trabajar con texto, números o fechas sin conversiones manuales.
* Esta flexibilidad facilita la integración de fuentes diversas (por ejemplo, IoT, CSV, logs).

Veracidad

La veracidad mide la calidad, confiabilidad y consistencia de los datos.
En Big Data, los errores, duplicados y valores nulos son frecuentes, y deben detectarse y corregirse para asegurar resultados válidos.

Ejemplo de análisis:
Contar valores duplicados en el dataset.

In [34]:
print(" V de Veracidad")

# Contar duplicados
duplicados = df.count() - df.dropDuplicates().count()
print(f"Número de registros duplicados: {duplicados}")


 V de Veracidad
Número de registros duplicados: 2


In [35]:
# Valores inconsistentes (ej. diferentes formas de escribir el mismo valor)
df.select("Departamento").groupBy("Departamento").count().orderBy(F.desc("count")).show(10)


+---------------+-----+
|   Departamento|count|
+---------------+-----+
|      Santander|   20|
|Valle del Cauca|   18|
|         Bogotá|   18|
|      Bogotá DC|   14|
|      Antioquia|   13|
|   Cundinamarca|   11|
|      Atlántico|    8|
+---------------+-----+



Interpretación

* Spark facilita la limpieza y estandarización de datos a gran escala.
* Permite detectar y corregir inconsistencias sin afectar el rendimiento.
* La veracidad garantiza que los análisis posteriores sean confiables.

Valor

El valor se refiere a la utilidad real de los datos para generar conocimiento o tomar decisiones. Un conjunto de datos tiene valor cuando permite extraer patrones, optimizar procesos o apoyar políticas basadas en evidencia.

Ejemplo de análisis:
Obtener información útil como promedios, distribuciones o tendencias.

In [52]:
print("ANÁLISIS DE VALOR")

# Agrupar por la columna Genero y contar cuántos registros hay por cada categoría
conteo_genero = (
    df_sin_nulls.groupBy("Genero")
    .count()
    .orderBy(F.desc("count"))
)

# Mostrar la tabla resumen
conteo_genero.show()

# Calcular el total de registros para obtener proporciones
total = df_sin_nulls.count()

# Agregar una nueva columna con el porcentaje que representa cada género
df_valor = conteo_genero.withColumn(
    "Porcentaje",
    (F.col("count") / total * 100).cast("double")
)

# Mostrar resultados finales
df_valor.show()

# Mostrar los resultados con una interpretación
print("Interpretación:")
print("Este análisis nos muestra la distribución de estudiantes por género.")
print("Con esta información podemos evaluar el equilibrio en los datos y detectar posibles sesgos.")

ANÁLISIS DE VALOR
+---------+-----+
|   Genero|count|
+---------+-----+
| FEMENINO|   56|
|MASCULINO|   41|
+---------+-----+

+---------+-----+-----------------+
|   Genero|count|       Porcentaje|
+---------+-----+-----------------+
| FEMENINO|   56|57.73195876288659|
|MASCULINO|   41| 42.2680412371134|
+---------+-----+-----------------+

Interpretación:
Este análisis nos muestra la distribución de estudiantes por género.
Con esta información podemos evaluar el equilibrio en los datos y detectar posibles sesgos.
